In [5]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

import numpy as np
import SimpleITK as sitk

In [6]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [7]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [8]:
dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [9]:
print(dataset[0])


['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3/1.2.840.113619.2.290.3.2500492204.268.1383804922.866.3', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101331015/1.2.124.113532.10.149.172.5.20131107.93455.7725981/1.2.840.113619.2.290.3.2500492204.268.1383804923.167/1.2.840.113619.2.290.3.2500492204.268.1383804923.167', 'PT', 'con_da_nov 7_2013.csv']


In [11]:
#voir pour gérer les liste de ceux qui ont un unconstant spacing ici 
liste_unconstant_spacing = []
for liste in dataset : 
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            print(liste)
            liste_unconstant_spacing.append(liste)
       

['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.3.46.670589.28.2.12.30.26421.36112.2.3508.0.1359080111/imagys_AHL2011_13011101431012_c72b9b8c-80b3-4803-a240-62a9ec6b6302', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431012/1.2.124.113532.172.16.25.2.20130121.814.1220583/1.2.840.113704.1.111.3652.1359035909.11/imagys_AHL2011_13011101431012_ce32ad94-1872-4340-9e14-a2b6ec451cf0', 'CT', 'can_sy_jan 24_2013.csv']
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391019/1.2.124.113532.172.16.4.13.20130416.213156.2050918/1.2.840.113619.2.290.3.279712783.8.1366178187.547/1.2.840.113619.2.290.3.279712783.8.1366178187.547', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101391019/1.2.124.113532.172.16.4.13.20130416.213156.2050918/1.2.840.113619.2.290.3.279712783.8.1366178186.947.3/1.2.840.113619.2.290.3.279712783.8.1366178186.947.3', 'CT', 'arb_al_apr 17_2013.csv']
['/media/salim/DD 2To/AHL2011_Validat

In [6]:

nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'


csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'


for serie in dataset : 
    print(serie)

    for i in range(1,4,2): #inde 1 et 3 => CT OU PT


        if serie[i] == 'CT' : 
            serie_ct_objet = SeriesCT(serie[i - 1])
            study_uid = serie_ct_objet.get_series_details()['study']['StudyInstanceUID']
            serie_ct_objet.get_numpy_array()
            if serie_ct_objet.get_z_spacing() != 'Unconstant Spacing' : 
                filename = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename))
                

            else : 
                print("Unconstant Spacing à la série CT " , dataset.index(serie))



    

        if serie[i] == 'PT' :
            print('serie PT, generation du mask avec fichier csv associée')
            serie_pt_objet = SeriesPT(serie[i - 1])
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing() != 'Unconstant Spacing' : 
                filename = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename))

                size_matrix = serie_pt_objet.get_size_matrix()
                print(size_matrix)

                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                #si le masque est ok, alors generer nifti :
                #filename = study_uid+'_'+'nifti'+'mask_4D'+'.nii'
                #serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename), mask_4D)
                print(mask_objet.calcul_suv(serie_pt_objet.get_numpy_array()))
                if mask_objet.is_correct_suv(serie_pt_objet.get_numpy_array()) == False : 
                    print("erreur calcul suv, philips")
                    print(mask_objet.ecart_suv_max(nifti_array))

            else : print("Unconstant Spacing à la série PT " , dataset.index(serie))



953.2.2148.0.1327586717/1.3.46.670589.28.2.12.21.6047.2953.2.2148.0.1327586717', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101431006/1.2.124.113532.172.16.25.2.20120125.456.1038547/1.2.840.113704.1.111.6380.1327585449.11/1.2.840.113704.1.111.6380.1327585449.11', 'CT', 'bou_ya_jan 26_2012.csv']
serie PT, generation du mask avec fichier csv associée
[144, 144, 213]
{1: {'SUV_max': 3.75, 'SUV_mean': 2.18, 'SD': 0.57}, 2: {'SUV_max': 3.39, 'SUV_mean': 2.12, 'SD': 0.52}, 3: {'SUV_max': 4.06, 'SUV_mean': 2.56, 'SD': 0.63}, 4: {'SUV_max': 2.32, 'SUV_mean': 1.29, 'SD': 0.31}, 5: {'SUV_max': 4.18, 'SUV_mean': 2.45, 'SD': 0.61}, 6: {'SUV_max': 4.41, 'SUV_mean': 2.34, 'SD': 0.54}, 7: {'SUV_max': 3.03, 'SUV_mean': 1.68, 'SD': 0.4}, 8: {'SUV_max': 6.97, 'SUV_mean': 3.98, 'SD': 0.99}, 9: {'SUV_max': 3.09, 'SUV_mean': 1.8, 'SD': 0.53}, 10: {'SUV_max': 2.55, 'SUV_mean': 1.63, 'SD': 0.36}, 11: {'SUV_max': 2.87, 'SUV_mean': 1.65, 'SD': 0.39}, 12: {'SUV_max': 4.94, 'SUV_mean': 2.84, 'SD': 

In [7]:
#pblm ici sur la méthode de calcul des SUV max mean, bcp trop long 
csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'
dataset[14]
objet = SeriesCT(dataset[14][0])
nifti = objet.get_numpy_array()
size_matrix = objet.get_size_matrix()
print(size_matrix)
mask_objet= MaskBuilder(os.path.join(csv_directory, dataset[14][4]), size_matrix)


[200, 200, 481]


In [3]:
import os 
from library_dicom.dicom_processor.model.csv_reader.CsvReader import CsvReader
csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'
filename = 'wil_ol_apr 24_2012.csv'

objet_csv_reader = CsvReader(os.path.join(csv_directory, filename))
objet_csv_reader .get_manual_rois()

[['Cervical',
  ' 1',
  ' 394',
  ' 425',
  ' num points = 6',
  ' 127 105',
  ' 115 94',
  ' 107 99',
  ' 108 106',
  ' 116 118',
  ' 125 112'],
 ['Cervical',
  ' 1',
  ' 378',
  ' 408',
  ' num points = 6',
  ' 124 110',
  ' 117 96',
  ' 107 97',
  ' 111 110',
  ' 120 120',
  ' 128 119'],
 ['Cervical',
  ' 11',
  ' 370',
  ' 384',
  ' num points = 3',
  ' 128 120',
  ' 132 120',
  ' 128 116'],
 ['Cervical',
  ' 11',
  ' 368',
  ' 386',
  ' num points = 3',
  ' 124 123',
  ' 128 123',
  ' 124 119'],
 ['Cervical',
  ' 1',
  ' 388',
  ' 426',
  ' num points = 7',
  ' 98 99',
  ' 94 97',
  ' 82 102',
  ' 78 118',
  ' 86 120',
  ' 93 108',
  ' 97 106'],
 ['Cervical',
  ' 1',
  ' 368',
  ' 406',
  ' num points = 6',
  ' 98 108',
  ' 89 108',
  ' 87 97',
  ' 94 95',
  ' 103 101',
  ' 103 104'],
 ['Cervical',
  ' 1',
  ' 368',
  ' 393',
  ' num points = 7',
  ' 90 123',
  ' 88 112',
  ' 84 109',
  ' 80 114',
  ' 76 124',
  ' 80 130',
  ' 87 129'],
 ['Mediastinal',
  ' 1',
  ' 349',
  ' 371',

In [ ]:
None

In [39]:
None

In [40]:
None

In [32]:
None

In [8]:
list_manufacturer = []
for serie in dataset : 
#serie CT / PT 
    for i in range(2):
        #print(i)
        #recupérer study IUD et CT PT dans le tri des données, dans le json 
        serie_object = Series(serie[i])
        serie_data = serie_object.get_series_details()
        modality = serie_data['series']['Modality'] #CT ou PT

        if modality == 'PT' :
            serie_pt = SeriesPT(serie[i])
            list_manufacturer.append(serie_pt.get_series_details()['series']['Manufacturer'])

print(list_manufacturer)

['GE MEDICAL SYSTEMS', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'GE MEDICAL SYSTEMS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'SIEMENS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'GE MEDICAL SYSTEMS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'CPS', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Philips Medical Systems', 'Phi